In [1]:
import pandas as pd
import numpy as np
import random
import os
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import *
from xgboost import *
from catboost import *
from mlxtend.classifier import StackingCVClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

import optuna
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [2]:
train = pd.read_csv('./data/train.csv', encoding='utf-8')
test = pd.read_csv('./data/test.csv', encoding='utf-8')
info = pd.read_csv('./data/snp_info.csv', encoding='utf-8')

In [3]:
train.shape, test.shape

((262, 21), (175, 20))

In [4]:
train

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,...,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A


In [5]:
train = train.drop(columns=['id'])
cols = train.columns.tolist()

### EDA

In [6]:
# prof = train.profile_report()

In [7]:
# prof

### 전처리

In [8]:
for i in cols:
    print('columns:',i, train[i].unique())

columns: father [0]
columns: mother [0]
columns: gender [0]
columns: trait [2 1]
columns: SNP_01 ['G G' 'A G' 'A A']
columns: SNP_02 ['A G' 'G G' 'A A']
columns: SNP_03 ['A A' 'C A' 'C C']
columns: SNP_04 ['G A' 'A A' 'G G']
columns: SNP_05 ['C A' 'A A' 'C C']
columns: SNP_06 ['A A' 'A G' 'G G']
columns: SNP_07 ['A A' 'G G' 'G A']
columns: SNP_08 ['G G' 'G A' 'A A']
columns: SNP_09 ['A A' 'G A' 'G G']
columns: SNP_10 ['G G' 'A G' 'A A']
columns: SNP_11 ['A G' 'A A' 'G G']
columns: SNP_12 ['A A' 'G A' 'G G']
columns: SNP_13 ['A A' 'G G' 'A G']
columns: SNP_14 ['A A' 'C C' 'C A']
columns: SNP_15 ['A A' 'G G' 'G A']
columns: class ['B' 'C' 'A']


In [9]:
# SNP 카테고리 확인

snp_list = []
for i in cols[4:-1]:
    for j in range(2):
        snp_list.append([k for k in train[i].unique().tolist()][j])

print(set(snp_list)) # 6개 카테고리

{'C C', 'C A', 'G A', 'A G', 'A A', 'G G'}


In [10]:
snp_list = []
for i in cols[4:-1]:
    for j in range(2):
        snp_list.append([k for k in test[i].unique().tolist()][j])

print(set(snp_list)) # 6개 카테고리

{'C C', 'C A', 'G A', 'A G', 'A A', 'G G'}


In [11]:
# V1과 전처리를 다르게 진행

test_id = test[['id']] # test ID 저장

target = train.iloc[:, -1]
train = train.iloc[:, 3:-1]
test = test.iloc[:, 4:]

train.shape, test.shape, target.shape

((262, 16), (175, 16), (262,))

In [12]:
for i in train.columns:
    print('Train :', i, train[i].unique())
    
for i in test.columns:
    print('Test :', i, test[i].unique())

Train : trait [2 1]
Train : SNP_01 ['G G' 'A G' 'A A']
Train : SNP_02 ['A G' 'G G' 'A A']
Train : SNP_03 ['A A' 'C A' 'C C']
Train : SNP_04 ['G A' 'A A' 'G G']
Train : SNP_05 ['C A' 'A A' 'C C']
Train : SNP_06 ['A A' 'A G' 'G G']
Train : SNP_07 ['A A' 'G G' 'G A']
Train : SNP_08 ['G G' 'G A' 'A A']
Train : SNP_09 ['A A' 'G A' 'G G']
Train : SNP_10 ['G G' 'A G' 'A A']
Train : SNP_11 ['A G' 'A A' 'G G']
Train : SNP_12 ['A A' 'G A' 'G G']
Train : SNP_13 ['A A' 'G G' 'A G']
Train : SNP_14 ['A A' 'C C' 'C A']
Train : SNP_15 ['A A' 'G G' 'G A']
Test : trait [1 2]
Test : SNP_01 ['A G' 'G G' 'A A']
Test : SNP_02 ['G G' 'A G' 'A A']
Test : SNP_03 ['A A' 'C C' 'C A']
Test : SNP_04 ['G A' 'G G' 'A A']
Test : SNP_05 ['A A' 'C C' 'C A']
Test : SNP_06 ['A G' 'A A' 'G G']
Test : SNP_07 ['G G' 'A A' 'G A']
Test : SNP_08 ['G A' 'A A' 'G G']
Test : SNP_09 ['G A' 'A A' 'G G']
Test : SNP_10 ['A G' 'G G' 'A A']
Test : SNP_11 ['A G' 'A A' 'G G']
Test : SNP_12 ['G A' 'A A' 'G G']
Test : SNP_13 ['G G' 'A A' '

In [13]:
# Label Encoding

encode_list = ['A A', 'A G', 'C A', 'C C', 'G A', 'G G']

for i, j in zip(encode_list, range(len(encode_list))):
    train = train.replace(i, int(j))
    test = test.replace(i, int(j))

In [14]:
print(set(target))
target_list = ['A', 'B', 'C']

for i, j in zip(target_list, range(len(target_list))):
    target = target.replace(i, int(j))
    
target

{'A', 'C', 'B'}


0      1
1      2
2      1
3      0
4      2
      ..
257    1
258    2
259    0
260    0
261    1
Name: class, Length: 262, dtype: int64

In [15]:
from collections import Counter

# Oversampling을 위한 값 설정

cnt = np.array(list(Counter(target).values())) # 1, 2, 0
cnt*2

array([228, 158, 138])

In [16]:
# 데이터가 적기 때문에 Oversampling 적용

smote = SMOTE(random_state=2023, sampling_strategy={0:138, 1:228, 2:158})

train_os, target_os = smote.fit_resample(train, target)

In [17]:
train_os.shape, target_os.shape

((524, 16), (524,))

### 모델 생성

In [18]:
knn = KNeighborsClassifier()
lgbm = LGBMClassifier(random_state=2023)
xgb = XGBClassifier(random_state=2023)
cb = CatBoostClassifier(random_state=2023)
rf = RandomForestClassifier(random_state=2023)

In [19]:
# Validation

tr, tr_val, tar, tar_val = train_test_split(train_os, target_os, stratify=target_os, random_state=2023)
tr.shape, tr_val.shape, tar.shape, tar_val.shape

((393, 16), (131, 16), (393,), (131,))

In [20]:
models = [knn, lgbm, xgb, cb, rf]
scores = []

for model in models:
    model.fit(tr, tar)
    scores.append(str(model)+' :')
    scores.append(f1_score(tar_val, model.predict(tr_val), average='macro'))
    
scores

Learning rate set to 0.075446
0:	learn: 1.0054543	total: 59ms	remaining: 59s
1:	learn: 0.9303438	total: 62.8ms	remaining: 31.3s
2:	learn: 0.8611801	total: 67.3ms	remaining: 22.4s
3:	learn: 0.8044084	total: 71.5ms	remaining: 17.8s
4:	learn: 0.7526894	total: 75.5ms	remaining: 15s
5:	learn: 0.7048407	total: 80.4ms	remaining: 13.3s
6:	learn: 0.6670772	total: 83.8ms	remaining: 11.9s
7:	learn: 0.6319221	total: 86.8ms	remaining: 10.8s
8:	learn: 0.5966940	total: 89.7ms	remaining: 9.87s
9:	learn: 0.5687875	total: 91.6ms	remaining: 9.06s
10:	learn: 0.5397929	total: 94.3ms	remaining: 8.48s
11:	learn: 0.5135210	total: 95.9ms	remaining: 7.89s
12:	learn: 0.4889549	total: 97.3ms	remaining: 7.39s
13:	learn: 0.4699599	total: 98.9ms	remaining: 6.97s
14:	learn: 0.4520232	total: 100ms	remaining: 6.59s
15:	learn: 0.4327281	total: 102ms	remaining: 6.25s
16:	learn: 0.4122205	total: 103ms	remaining: 5.95s
17:	learn: 0.3970760	total: 104ms	remaining: 5.68s
18:	learn: 0.3827771	total: 105ms	remaining: 5.43s
19:

345:	learn: 0.0141643	total: 345ms	remaining: 652ms
346:	learn: 0.0141203	total: 346ms	remaining: 651ms
347:	learn: 0.0140768	total: 347ms	remaining: 649ms
348:	learn: 0.0140411	total: 347ms	remaining: 648ms
349:	learn: 0.0140050	total: 348ms	remaining: 647ms
350:	learn: 0.0139561	total: 349ms	remaining: 645ms
351:	learn: 0.0139109	total: 350ms	remaining: 644ms
352:	learn: 0.0138698	total: 350ms	remaining: 642ms
353:	learn: 0.0138381	total: 351ms	remaining: 641ms
354:	learn: 0.0138022	total: 352ms	remaining: 639ms
355:	learn: 0.0137470	total: 353ms	remaining: 638ms
356:	learn: 0.0137122	total: 353ms	remaining: 636ms
357:	learn: 0.0136719	total: 354ms	remaining: 635ms
358:	learn: 0.0136333	total: 355ms	remaining: 634ms
359:	learn: 0.0135846	total: 356ms	remaining: 632ms
360:	learn: 0.0135386	total: 356ms	remaining: 631ms
361:	learn: 0.0134869	total: 357ms	remaining: 629ms
362:	learn: 0.0134061	total: 358ms	remaining: 628ms
363:	learn: 0.0133536	total: 359ms	remaining: 626ms
364:	learn: 

570:	learn: 0.0076450	total: 515ms	remaining: 387ms
571:	learn: 0.0076315	total: 516ms	remaining: 386ms
572:	learn: 0.0076096	total: 517ms	remaining: 385ms
573:	learn: 0.0075902	total: 518ms	remaining: 384ms
574:	learn: 0.0075699	total: 519ms	remaining: 383ms
575:	learn: 0.0075525	total: 519ms	remaining: 382ms
576:	learn: 0.0075363	total: 520ms	remaining: 381ms
577:	learn: 0.0075188	total: 521ms	remaining: 380ms
578:	learn: 0.0074996	total: 521ms	remaining: 379ms
579:	learn: 0.0074829	total: 522ms	remaining: 378ms
580:	learn: 0.0074680	total: 523ms	remaining: 377ms
581:	learn: 0.0074520	total: 523ms	remaining: 376ms
582:	learn: 0.0074409	total: 524ms	remaining: 375ms
583:	learn: 0.0074265	total: 525ms	remaining: 374ms
584:	learn: 0.0074118	total: 526ms	remaining: 373ms
585:	learn: 0.0073969	total: 526ms	remaining: 372ms
586:	learn: 0.0073797	total: 527ms	remaining: 371ms
587:	learn: 0.0073663	total: 528ms	remaining: 370ms
588:	learn: 0.0073498	total: 529ms	remaining: 369ms
589:	learn: 

811:	learn: 0.0050429	total: 684ms	remaining: 158ms
812:	learn: 0.0050339	total: 684ms	remaining: 157ms
813:	learn: 0.0050276	total: 685ms	remaining: 157ms
814:	learn: 0.0050195	total: 686ms	remaining: 156ms
815:	learn: 0.0050142	total: 687ms	remaining: 155ms
816:	learn: 0.0050080	total: 687ms	remaining: 154ms
817:	learn: 0.0050022	total: 688ms	remaining: 153ms
818:	learn: 0.0049955	total: 689ms	remaining: 152ms
819:	learn: 0.0049879	total: 689ms	remaining: 151ms
820:	learn: 0.0049802	total: 690ms	remaining: 150ms
821:	learn: 0.0049760	total: 691ms	remaining: 150ms
822:	learn: 0.0049696	total: 691ms	remaining: 149ms
823:	learn: 0.0049632	total: 692ms	remaining: 148ms
824:	learn: 0.0049556	total: 693ms	remaining: 147ms
825:	learn: 0.0049502	total: 694ms	remaining: 146ms
826:	learn: 0.0049416	total: 694ms	remaining: 145ms
827:	learn: 0.0049337	total: 695ms	remaining: 144ms
828:	learn: 0.0049283	total: 696ms	remaining: 143ms
829:	learn: 0.0049207	total: 696ms	remaining: 143ms
830:	learn: 

['KNeighborsClassifier() :',
 0.9644101999633095,
 'LGBMClassifier(random_state=2023) :',
 0.9501742799486332,
 "XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,\n              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,\n              early_stopping_rounds=None, enable_categorical=False,\n              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,\n              grow_policy='depthwise', importance_type=None,\n              interaction_constraints='', learning_rate=0.300000012,\n              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,\n              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,\n              missing=nan, monotone_constraints='()', n_estimators=100,\n              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',\n              predictor='auto', ...) :",
 0.9644101999633095,
 '<catboost.core.CatBoostClassifier object at 0x000001E8E6EF8070> :',
 0.9571175950486296,
 'RandomForestClassif

In [21]:
from sklearn.model_selection import RepeatedKFold, cross_val_score

for model in models:
    rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=2023)
    score = cross_val_score(model, train_os, target_os, scoring='f1_macro', cv=rkf, n_jobs=-1)
    print(str(model),':', score.mean())

KNeighborsClassifier() : 0.9590831176841145
LGBMClassifier(random_state=2023) : 0.9734329623945237
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...) : 0.9696587665684882
<catboost.core.CatBoostClassifier object at 0x000001E8E6EF8070> : 0.9785987197741298
RandomForestClassifier(random_state=2023) : 0.978316273

### 튜닝

In [22]:
# KNN
prs = []

for i in range(2, 25):
    knn = KNeighborsClassifier(n_neighbors=i)
    score = cross_val_score(knn, train_os, target_os, scoring='f1_macro', cv=rkf, n_jobs=-1)
    print('n_neighbors :', i, score.mean())
    prs.append([i, score.mean()])

np.argmax([prs[i][1] for i in range(len(prs))])

# n_neighbors = 3

n_neighbors : 2 0.9556027275150635
n_neighbors : 3 0.9595922686845619
n_neighbors : 4 0.9555627125922223
n_neighbors : 5 0.9590831176841145
n_neighbors : 6 0.9561695338359344
n_neighbors : 7 0.9574206673164967
n_neighbors : 8 0.9511029695449317
n_neighbors : 9 0.9558818998520872
n_neighbors : 10 0.944937842569997
n_neighbors : 11 0.9482368721389708
n_neighbors : 12 0.9361462608123545
n_neighbors : 13 0.9419714756204884
n_neighbors : 14 0.9255396440777224
n_neighbors : 15 0.9372582252229644
n_neighbors : 16 0.928008600771442
n_neighbors : 17 0.933538143840982
n_neighbors : 18 0.9276401586284843
n_neighbors : 19 0.937656994003776
n_neighbors : 20 0.9293665673796542
n_neighbors : 21 0.936126474554583
n_neighbors : 22 0.932445708999193
n_neighbors : 23 0.938628899930125
n_neighbors : 24 0.9333782468689641


1

In [23]:
# XGBoost 

def xgbtuner(trial, data=tr, target=tar):
    
    param = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_int('max_depth', 3,30),
        'min_child_weight': trial.suggest_categorical('min_child_weight', [1, 3, 5]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'n_estimators' : trial.suggest_categorical('n_estimators', [100,200,300,400,500]),
        'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
        'random_state': 2023
    }
    model = XGBClassifier(**param)
    
    model.fit(tr, tar)
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(xgbtuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_xgb = XGBClassifier(**study.best_params)

# xgb_params = {'learning_rate': 0.008, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.6, 'colsample_bytree': 0.3, 'n_estimators': 200, 'objective': 'reg:squarederror'}

[I 2023-01-06 00:14:21,521] A new study created in memory with name: no-name-059deefa-d45e-4765-93d5-22087a1aa776
[I 2023-01-06 00:14:21,811] Trial 0 finished with value: 0.9711567053226288 and parameters: {'learning_rate': 0.008, 'max_depth': 11, 'min_child_weight': 3, 'subsample': 0.5, 'colsample_bytree': 0.6, 'n_estimators': 200, 'objective': 'reg:squarederror'}. Best is trial 0 with value: 0.9711567053226288.
[I 2023-01-06 00:14:21,951] Trial 1 finished with value: 0.920298786181139 and parameters: {'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 5, 'subsample': 1.0, 'colsample_bytree': 1.0, 'n_estimators': 100, 'objective': 'reg:squarederror'}. Best is trial 0 with value: 0.9711567053226288.
[I 2023-01-06 00:14:22,554] Trial 2 finished with value: 0.9644101999633095 and parameters: {'learning_rate': 0.006, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.5, 'colsample_bytree': 1.0, 'n_estimators': 400, 'objective': 'reg:squarederror'}. Best is trial 0 with value: 0

[I 2023-01-06 00:14:33,932] Trial 26 finished with value: 0.9711567053226288 and parameters: {'learning_rate': 0.017, 'max_depth': 26, 'min_child_weight': 5, 'subsample': 0.4, 'colsample_bytree': 0.3, 'n_estimators': 200, 'objective': 'reg:squarederror'}. Best is trial 3 with value: 0.9784659784659785.
[I 2023-01-06 00:14:34,624] Trial 27 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.01, 'max_depth': 20, 'min_child_weight': 3, 'subsample': 0.5, 'colsample_bytree': 0.4, 'n_estimators': 500, 'objective': 'reg:squarederror'}. Best is trial 3 with value: 0.9784659784659785.
[I 2023-01-06 00:14:35,282] Trial 28 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.014, 'max_depth': 12, 'min_child_weight': 3, 'subsample': 0.5, 'colsample_bytree': 0.4, 'n_estimators': 500, 'objective': 'reg:squarederror'}. Best is trial 3 with value: 0.9784659784659785.
[I 2023-01-06 00:14:35,820] Trial 29 finished with value: 0.9784659784659785 and paramete

Number of finished trials: 50
Best trial: {'learning_rate': 0.014, 'max_depth': 23, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.4, 'n_estimators': 300, 'objective': 'reg:squarederror'}


In [24]:
# CatBoost

def cattuner(trial):
    param = {
                #'iterations' : 10000, replaced by early stopping
                'eval_metric': 'TotalF1',
                'use_best_model': True,
                'random_seed' : 2023,
                'learning_rate' :trial.suggest_loguniform('learning_rate', 0.01, 0.3),
                "depth": trial.suggest_int("depth", 1, 15),
                'l2_leaf_reg' :trial.suggest_loguniform('l2_leaf_reg', 1e-8, 20),
                'random_strength' : trial.suggest_loguniform('random_strength', 1, 50),
                'grow_policy':trial.suggest_categorical ('grow_policy', ['Lossguide','SymmetricTree']),
                'max_bin': trial.suggest_int("max_bin", 20, 500),
                'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 1, 100),
                "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"])
            }
    
    if param['grow_policy'] == 'Lossguide':
        param['max_leaves']:trial.suggest_int('max_leaves', 1, 100)
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    model = CatBoostClassifier(**param)
    model.fit(tr, tar, eval_set=[(tr_val, tar_val)], verbose=0, early_stopping_rounds=100)
    
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(cattuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_cb = CatBoostClassifier(**study.best_params)

# cb_params = {'learning_rate': 0.19682921840168557, 'depth': 6, 'l2_leaf_reg': 8.56339618023411e-06, 'random_strength': 3.406192048764729, 'grow_policy': 'SymmetricTree', 'max_bin': 165, 'min_data_in_leaf': 45, 'bootstrap_type': 'Bernoulli', 'subsample': 0.5641166296761003}

[I 2023-01-06 00:14:45,123] A new study created in memory with name: no-name-b7ec9471-70f4-4613-840a-448095cc0892
[I 2023-01-06 00:14:45,478] Trial 0 finished with value: 0.9711567053226288 and parameters: {'learning_rate': 0.0918233789714613, 'depth': 2, 'l2_leaf_reg': 9.574973005474794e-07, 'random_strength': 4.324186472143466, 'grow_policy': 'SymmetricTree', 'max_bin': 321, 'min_data_in_leaf': 31, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.2352008705331325}. Best is trial 0 with value: 0.9711567053226288.
[I 2023-01-06 00:14:45,959] Trial 1 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.017909849337039336, 'depth': 11, 'l2_leaf_reg': 4.057121865688517, 'random_strength': 1.168913896488504, 'grow_policy': 'Lossguide', 'max_bin': 72, 'min_data_in_leaf': 55, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.6480499361790475}. Best is trial 1 with value: 0.9784659784659785.
[I 2023-01-06 00:14:46,391] Trial 2 finished with value: 0.978465978

[I 2023-01-06 00:15:03,573] Trial 20 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.22529082450430982, 'depth': 11, 'l2_leaf_reg': 1.825490318842189e-07, 'random_strength': 23.661096195016086, 'grow_policy': 'Lossguide', 'max_bin': 21, 'min_data_in_leaf': 19, 'bootstrap_type': 'Bernoulli', 'subsample': 0.1291483723433383}. Best is trial 13 with value: 0.9928820399926619.
[I 2023-01-06 00:15:04,121] Trial 21 finished with value: 0.9716374269005849 and parameters: {'learning_rate': 0.20576637140284873, 'depth': 4, 'l2_leaf_reg': 0.0015190128561841714, 'random_strength': 2.4988351235898536, 'grow_policy': 'SymmetricTree', 'max_bin': 285, 'min_data_in_leaf': 16, 'bootstrap_type': 'Bernoulli', 'subsample': 0.7568267941080122}. Best is trial 13 with value: 0.9928820399926619.
[I 2023-01-06 00:15:04,621] Trial 22 finished with value: 0.9857058650162099 and parameters: {'learning_rate': 0.010605914044413231, 'depth': 7, 'l2_leaf_reg': 4.720313250399367e-06, 'random

[I 2023-01-06 00:15:13,557] Trial 40 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.019777990977792904, 'depth': 3, 'l2_leaf_reg': 7.657978497968932e-08, 'random_strength': 2.071341205460429, 'grow_policy': 'Lossguide', 'max_bin': 101, 'min_data_in_leaf': 29, 'bootstrap_type': 'Bernoulli', 'subsample': 0.38746939060794316}. Best is trial 13 with value: 0.9928820399926619.
[I 2023-01-06 00:15:14,033] Trial 41 finished with value: 0.9784659784659785 and parameters: {'learning_rate': 0.197860576263204, 'depth': 11, 'l2_leaf_reg': 1.0188195419622975e-08, 'random_strength': 23.06569289098054, 'grow_policy': 'Lossguide', 'max_bin': 29, 'min_data_in_leaf': 43, 'bootstrap_type': 'Bernoulli', 'subsample': 0.10828579669777945}. Best is trial 13 with value: 0.9928820399926619.
[I 2023-01-06 00:15:14,526] Trial 42 finished with value: 0.9928820399926619 and parameters: {'learning_rate': 0.23942305310248505, 'depth': 10, 'l2_leaf_reg': 2.2698692945232533e-07, 'random_st

Number of finished trials: 50
Best trial: {'learning_rate': 0.19682921840168557, 'depth': 6, 'l2_leaf_reg': 8.56339618023411e-06, 'random_strength': 3.406192048764729, 'grow_policy': 'SymmetricTree', 'max_bin': 165, 'min_data_in_leaf': 45, 'bootstrap_type': 'Bernoulli', 'subsample': 0.5641166296761003}


In [25]:
# LightGBM

def lgbtuner(trial,data=tr,target=tar):
    
    param = {
        'metric': 'multi_error', 
        'random_state': 2023,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100),
        'random_state' : 2023
    }
    model = LGBMClassifier(**param)
    
    model.fit(tr,tar,eval_set=[(tr_val,tar_val)], verbose=0, early_stopping_rounds= 100)
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(lgbtuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_lgb = LGBMClassifier(**study.best_params) 

# lgb_params = {'reg_alpha': 0.16768226852129442, 'reg_lambda': 0.018910145981297394, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 0.006, 'max_depth': 10, 'num_leaves': 783}

[I 2023-01-06 00:15:17,408] A new study created in memory with name: no-name-e5bb5acf-9d9a-46dc-a44b-b17305cc0342
[I 2023-01-06 00:15:17,547] Trial 0 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.34630840688371006, 'reg_lambda': 0.5907244199346164, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.017, 'max_depth': 100, 'num_leaves': 33, 'min_data_per_groups': 75}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-06 00:15:17,831] Trial 1 finished with value: 0.9711567053226288 and parameters: {'reg_alpha': 7.296630134081589, 'reg_lambda': 0.009229512224534409, 'colsample_bytree': 0.6, 'subsample': 0.7, 'learning_rate': 0.017, 'max_depth': 100, 'num_leaves': 770, 'min_data_per_groups': 100}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-06 00:15:18,081] Trial 2 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.026966163917108054, 'reg_lambda': 0.07102279020568572, 'colsample_bytree': 0.3, 'subsample': 0.8

[I 2023-01-06 00:15:24,055] Trial 23 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.13540642961627383, 'reg_lambda': 0.38694228744139214, 'colsample_bytree': 0.4, 'subsample': 0.7, 'learning_rate': 0.017, 'max_depth': 20, 'num_leaves': 187, 'min_data_per_groups': 4}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-06 00:15:24,430] Trial 24 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.007305360648737183, 'reg_lambda': 3.3025034366540975, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 0.01, 'max_depth': 10, 'num_leaves': 228, 'min_data_per_groups': 18}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-06 00:15:24,723] Trial 25 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.7764744193140579, 'reg_lambda': 0.0010590734870382231, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 20, 'num_leaves': 998, 'min_data_per_groups': 59}. Best is trial 0 with va

[I 2023-01-06 00:15:30,145] Trial 47 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 1.619966934000536, 'reg_lambda': 0.001217042609565819, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.014, 'max_depth': 20, 'num_leaves': 976, 'min_data_per_groups': 60}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-06 00:15:30,517] Trial 48 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 0.00836366473984989, 'reg_lambda': 0.3400945577964181, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.006, 'max_depth': 20, 'num_leaves': 276, 'min_data_per_groups': 47}. Best is trial 0 with value: 0.9784659784659785.
[I 2023-01-06 00:15:30,854] Trial 49 finished with value: 0.9784659784659785 and parameters: {'reg_alpha': 1.9635377306542432, 'reg_lambda': 0.0011021397517655147, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.014, 'max_depth': 20, 'num_leaves': 892, 'min_data_per_groups': 59}. Best is trial 0 with va

Number of finished trials: 50
Best trial: {'reg_alpha': 0.34630840688371006, 'reg_lambda': 0.5907244199346164, 'colsample_bytree': 1.0, 'subsample': 0.7, 'learning_rate': 0.017, 'max_depth': 100, 'num_leaves': 33, 'min_data_per_groups': 75}


In [26]:
# RandomForest

def rftuner(trial,data=tr,target=tar):
    
    param = {
        'max_depth':trial.suggest_int('max_depth', 3,30), 
        'min_samples_split':trial.suggest_int('min_samples_split',2, 20), 
        'min_samples_leaf':trial.suggest_int('min_samples_leaf',1, 10),
        'max_samples':trial.suggest_int('max_samples',1,10),
        'n_estimators':trial.suggest_categorical('n_estimators', [100,200,300,400,500]),
        'max_features':trial.suggest_categorical('max_features',['auto','log2']),
        'random_state':2023
    }
    
    model = RandomForestClassifier(**param)
    
    model.fit(tr,tar)
    preds = model.predict(tr_val)
    f1score = f1_score(tar_val, preds, average='macro')
    
    return f1score

study = optuna.create_study(direction='maximize')
study.optimize(rftuner, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

tuned_rf = RandomForestClassifier(**study.best_params) 

# rf_params = {'max_depth': 7, 'min_samples_split': 4, 'min_samples_leaf': 1,
#              'max_samples': 9, 'n_estimators': 500, 'max_features': 'log2'}

[I 2023-01-06 00:15:30,893] A new study created in memory with name: no-name-3241e307-382b-4992-a6d0-0d2fa4b76fce
[I 2023-01-06 00:15:31,069] Trial 0 finished with value: 0.2021276595744681 and parameters: {'max_depth': 19, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_samples': 4, 'n_estimators': 100, 'max_features': 'auto'}. Best is trial 0 with value: 0.2021276595744681.
[I 2023-01-06 00:15:31,207] Trial 1 finished with value: 0.2021276595744681 and parameters: {'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_samples': 8, 'n_estimators': 100, 'max_features': 'log2'}. Best is trial 0 with value: 0.2021276595744681.
[I 2023-01-06 00:15:31,736] Trial 2 finished with value: 0.2021276595744681 and parameters: {'max_depth': 10, 'min_samples_split': 19, 'min_samples_leaf': 6, 'max_samples': 2, 'n_estimators': 400, 'max_features': 'log2'}. Best is trial 0 with value: 0.2021276595744681.
[I 2023-01-06 00:15:32,126] Trial 3 finished with value: 0.2021276595744681 a

[I 2023-01-06 00:15:46,588] Trial 29 finished with value: 0.2021276595744681 and parameters: {'max_depth': 18, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_samples': 1, 'n_estimators': 500, 'max_features': 'auto'}. Best is trial 16 with value: 0.8577712609970675.
[I 2023-01-06 00:15:47,130] Trial 30 finished with value: 0.2021276595744681 and parameters: {'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 7, 'max_samples': 5, 'n_estimators': 400, 'max_features': 'auto'}. Best is trial 16 with value: 0.8577712609970675.
[I 2023-01-06 00:15:47,829] Trial 31 finished with value: 0.7885328101155439 and parameters: {'max_depth': 23, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_samples': 9, 'n_estimators': 500, 'max_features': 'auto'}. Best is trial 16 with value: 0.8577712609970675.
[I 2023-01-06 00:15:48,537] Trial 32 finished with value: 0.8295040803515379 and parameters: {'max_depth': 24, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_samples': 9, 'n_estimato

Number of finished trials: 50
Best trial: {'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_samples': 10, 'n_estimators': 300, 'max_features': 'log2'}


### 최종 모델

In [37]:
tuned_knn = KNeighborsClassifier(n_neighbors=3)

from tqdm import tqdm

for i in tqdm(range(4)):
    estimators = [('xgb', tuned_xgb), ('lgb', tuned_lgb), ('cb', tuned_cb), ('knn', tuned_knn)]
    fin_estimator = [tuned_xgb, tuned_lgb, tuned_cb, tuned_knn]
    
    est_list = estimators.pop(i)
    fin_est = fin_estimator.pop(i)
    
    stacks = StackingClassifier(estimators=estimators, final_estimator=fin_est, stack_method='predict_proba', cv=10)
    stacks.fit(tr, tar)
    
    pred = stacks.predict(tr_val)
    
    rkf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=2023)
    score = cross_val_score(stacks, train_os, target_os, scoring='f1_macro', cv=rkf, n_jobs=-1)
    
    print(f'Final Estimator : {fin_est} Model Score :', score.mean())    

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0:	learn: 1.0219772	total: 535us	remaining: 534ms
1:	learn: 0.7484596	total: 1.41ms	remaining: 705ms
2:	learn: 0.6188189	total: 2.29ms	remaining: 762ms
3:	learn: 0.5307641	total: 3.15ms	remaining: 784ms
4:	learn: 0.4414215	total: 3.96ms	remaining: 788ms
5:	learn: 0.3807869	total: 4.84ms	remaining: 802ms
6:	learn: 0.3322537	total: 5.69ms	remaining: 807ms
7:	learn: 0.2826099	total: 6.59ms	remaining: 817ms
8:	learn: 0.2525198	total: 7.37ms	remaining: 812ms
9:	learn: 0.2246188	total: 8.25ms	remaining: 817ms
10:	learn: 0.1994575	total: 9.04ms	remaining: 813ms
11:	learn: 0.1773414	total: 9.87ms	remaining: 813ms
12:	learn: 0.1562060	total: 10.6ms	remaining: 808ms
13:	learn: 0.1469026	total: 11.1ms	remaining: 785ms
14:	learn: 0.1440103	total: 11.6ms	remaining: 759ms
15:	learn: 0.1307031	total: 12.3ms	remaining: 756ms
16:	learn: 0.1190558	total: 13ms	remaining: 753ms
17:	learn: 0.1066914	total: 13.8ms	remaining: 752ms
18:	learn: 0.0946

279:	learn: 0.0000000	total: 225ms	remaining: 578ms
280:	learn: 0.0000000	total: 226ms	remaining: 578ms
281:	learn: 0.0000000	total: 227ms	remaining: 578ms
282:	learn: 0.0000000	total: 228ms	remaining: 578ms
283:	learn: 0.0000000	total: 229ms	remaining: 578ms
284:	learn: 0.0000000	total: 230ms	remaining: 577ms
285:	learn: 0.0000000	total: 231ms	remaining: 576ms
286:	learn: 0.0000000	total: 232ms	remaining: 575ms
287:	learn: 0.0000000	total: 232ms	remaining: 575ms
288:	learn: 0.0000000	total: 233ms	remaining: 574ms
289:	learn: 0.0000000	total: 234ms	remaining: 573ms
290:	learn: 0.0000000	total: 235ms	remaining: 572ms
291:	learn: 0.0000000	total: 236ms	remaining: 571ms
292:	learn: 0.0000000	total: 236ms	remaining: 570ms
293:	learn: 0.0000000	total: 237ms	remaining: 569ms
294:	learn: 0.0000000	total: 238ms	remaining: 569ms
295:	learn: 0.0000000	total: 239ms	remaining: 568ms
296:	learn: 0.0000000	total: 240ms	remaining: 568ms
297:	learn: 0.0000000	total: 241ms	remaining: 567ms
298:	learn: 

501:	learn: 0.0000000	total: 401ms	remaining: 397ms
502:	learn: 0.0000000	total: 402ms	remaining: 397ms
503:	learn: 0.0000000	total: 402ms	remaining: 396ms
504:	learn: 0.0000000	total: 403ms	remaining: 395ms
505:	learn: 0.0000000	total: 404ms	remaining: 394ms
506:	learn: 0.0000000	total: 405ms	remaining: 394ms
507:	learn: 0.0000000	total: 406ms	remaining: 393ms
508:	learn: 0.0000000	total: 406ms	remaining: 392ms
509:	learn: 0.0000000	total: 407ms	remaining: 391ms
510:	learn: 0.0000000	total: 408ms	remaining: 390ms
511:	learn: 0.0000000	total: 409ms	remaining: 390ms
512:	learn: 0.0000000	total: 410ms	remaining: 389ms
513:	learn: 0.0000000	total: 410ms	remaining: 388ms
514:	learn: 0.0000000	total: 411ms	remaining: 387ms
515:	learn: 0.0000000	total: 412ms	remaining: 386ms
516:	learn: 0.0000000	total: 413ms	remaining: 386ms
517:	learn: 0.0000000	total: 414ms	remaining: 385ms
518:	learn: 0.0000000	total: 414ms	remaining: 384ms
519:	learn: 0.0000000	total: 415ms	remaining: 383ms
520:	learn: 

730:	learn: 0.0000000	total: 573ms	remaining: 211ms
731:	learn: 0.0000000	total: 574ms	remaining: 210ms
732:	learn: 0.0000000	total: 575ms	remaining: 210ms
733:	learn: 0.0000000	total: 576ms	remaining: 209ms
734:	learn: 0.0000000	total: 577ms	remaining: 208ms
735:	learn: 0.0000000	total: 577ms	remaining: 207ms
736:	learn: 0.0000000	total: 578ms	remaining: 206ms
737:	learn: 0.0000000	total: 579ms	remaining: 206ms
738:	learn: 0.0000000	total: 580ms	remaining: 205ms
739:	learn: 0.0000000	total: 580ms	remaining: 204ms
740:	learn: 0.0000000	total: 581ms	remaining: 203ms
741:	learn: 0.0000000	total: 582ms	remaining: 202ms
742:	learn: 0.0000000	total: 583ms	remaining: 202ms
743:	learn: 0.0000000	total: 583ms	remaining: 201ms
744:	learn: 0.0000000	total: 584ms	remaining: 200ms
745:	learn: 0.0000000	total: 585ms	remaining: 199ms
746:	learn: 0.0000000	total: 586ms	remaining: 198ms
747:	learn: 0.0000000	total: 587ms	remaining: 198ms
748:	learn: 0.0000000	total: 588ms	remaining: 197ms
749:	learn: 

...

480:	learn: 0.0000000	total: 339ms	remaining: 366ms
481:	learn: 0.0000000	total: 340ms	remaining: 365ms
482:	learn: 0.0000000	total: 341ms	remaining: 365ms
483:	learn: 0.0000000	total: 341ms	remaining: 364ms
484:	learn: 0.0000000	total: 342ms	remaining: 363ms
485:	learn: 0.0000000	total: 343ms	remaining: 362ms
486:	learn: 0.0000000	total: 343ms	remaining: 362ms
487:	learn: 0.0000000	total: 344ms	remaining: 361ms
488:	learn: 0.0000000	total: 345ms	remaining: 360ms
489:	learn: 0.0000000	total: 345ms	remaining: 359ms
490:	learn: 0.0000000	total: 346ms	remaining: 359ms
491:	learn: 0.0000000	total: 347ms	remaining: 358ms
492:	learn: 0.0000000	total: 348ms	remaining: 357ms
493:	learn: 0.0000000	total: 348ms	remaining: 357ms
494:	learn: 0.0000000	total: 349ms	remaining: 356ms
495:	learn: 0.0000000	total: 350ms	remaining: 355ms
496:	learn: 0.0000000	total: 350ms	remaining: 354ms
497:	learn: 0.0000000	total: 351ms	remaining: 354ms
498:	learn: 0.0000000	total: 352ms	remaining: 353ms
499:	learn: 

722:	learn: 0.0000000	total: 508ms	remaining: 195ms
723:	learn: 0.0000000	total: 509ms	remaining: 194ms
724:	learn: 0.0000000	total: 510ms	remaining: 193ms
725:	learn: 0.0000000	total: 510ms	remaining: 193ms
726:	learn: 0.0000000	total: 511ms	remaining: 192ms
727:	learn: 0.0000000	total: 512ms	remaining: 191ms
728:	learn: 0.0000000	total: 512ms	remaining: 190ms
729:	learn: 0.0000000	total: 513ms	remaining: 190ms
730:	learn: 0.0000000	total: 514ms	remaining: 189ms
731:	learn: 0.0000000	total: 514ms	remaining: 188ms
732:	learn: 0.0000000	total: 515ms	remaining: 188ms
733:	learn: 0.0000000	total: 516ms	remaining: 187ms
734:	learn: 0.0000000	total: 516ms	remaining: 186ms
735:	learn: 0.0000000	total: 517ms	remaining: 185ms
736:	learn: 0.0000000	total: 518ms	remaining: 185ms
737:	learn: 0.0000000	total: 519ms	remaining: 184ms
738:	learn: 0.0000000	total: 519ms	remaining: 183ms
739:	learn: 0.0000000	total: 520ms	remaining: 183ms
740:	learn: 0.0000000	total: 521ms	remaining: 182ms
741:	learn: 

961:	learn: 0.0000000	total: 677ms	remaining: 26.7ms
962:	learn: 0.0000000	total: 677ms	remaining: 26ms
963:	learn: 0.0000000	total: 678ms	remaining: 25.3ms
964:	learn: 0.0000000	total: 679ms	remaining: 24.6ms
965:	learn: 0.0000000	total: 680ms	remaining: 23.9ms
966:	learn: 0.0000000	total: 680ms	remaining: 23.2ms
967:	learn: 0.0000000	total: 681ms	remaining: 22.5ms
968:	learn: 0.0000000	total: 682ms	remaining: 21.8ms
969:	learn: 0.0000000	total: 682ms	remaining: 21.1ms
970:	learn: 0.0000000	total: 683ms	remaining: 20.4ms
971:	learn: 0.0000000	total: 684ms	remaining: 19.7ms
972:	learn: 0.0000000	total: 684ms	remaining: 19ms
973:	learn: 0.0000000	total: 685ms	remaining: 18.3ms
974:	learn: 0.0000000	total: 686ms	remaining: 17.6ms
975:	learn: 0.0000000	total: 687ms	remaining: 16.9ms
976:	learn: 0.0000000	total: 687ms	remaining: 16.2ms
977:	learn: 0.0000000	total: 688ms	remaining: 15.5ms
978:	learn: 0.0000000	total: 689ms	remaining: 14.8ms
979:	learn: 0.0000000	total: 689ms	remaining: 14.1

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [09:06<00:00, 136.56s/it]

Final Estimator : KNeighborsClassifier(n_neighbors=3) Model Score : 0.9764019725049681


In [30]:
# Final Estimator - XGBoost

estimators = [('rf', tuned_rf), ('lgb', tuned_lgb(verbose=0)), ('cb', tuned_cb), ('knn', tuned_knn)]
fin_estimator = tuned_xgb
    
stacks = StackingClassifier(estimators=estimators, final_estimator=fin_estimator, stack_method='predict_proba', cv=10)
stacks.fit(train_os, target_os)
    
preds = stacks.predict(test)

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0:	learn: 0.8286292	total: 1.08ms	remaining: 1.08s
1:	learn: 0.6610882	total: 1.98ms	remaining: 989ms
2:	learn: 0.5664540	total: 2.92ms	remaining: 969ms
3:	learn: 0.4836907	total: 3.63ms	remaining: 904ms
4:	learn: 0.4055795	total: 4.51ms	remaining: 898ms
5:	learn: 0.3437752	total: 5.47ms	remaining: 906ms
6:	learn: 0.2994826	total: 6.43ms	remaining: 912ms
7:	learn: 0.2685972	total: 7.14ms	remaining: 885ms
8:	learn: 0.2381159	total: 8ms	remaining: 881ms
9:	learn: 0.2103328	total: 8.94ms	remaining: 885ms
10:	learn: 0.1890115	total: 9.84ms	remaining: 885ms
11:	learn: 0.1719544	total: 10.7ms	remaining: 877ms
12:	learn: 0.1568414	total: 11.5ms	remaining: 872ms
13:	learn: 0.1425141	total: 12.3ms	remaining: 868ms
14:	learn: 0.1295124	total: 13.2ms	remaining: 866ms
15:	learn: 0.1165083	total: 14.1ms	remaining: 866ms
16:	learn: 0.1078098	total: 15.5ms	remaining: 894ms
17:	learn: 0.0983576	total: 16.2ms	remaining: 886ms
18:	learn: 0.0924

216:	learn: 0.0000004	total: 191ms	remaining: 690ms
217:	learn: 0.0000004	total: 192ms	remaining: 689ms
218:	learn: 0.0000004	total: 193ms	remaining: 688ms
219:	learn: 0.0000004	total: 194ms	remaining: 688ms
220:	learn: 0.0000004	total: 195ms	remaining: 687ms
221:	learn: 0.0000003	total: 196ms	remaining: 687ms
222:	learn: 0.0000003	total: 197ms	remaining: 686ms
223:	learn: 0.0000003	total: 198ms	remaining: 685ms
224:	learn: 0.0000003	total: 199ms	remaining: 685ms
225:	learn: 0.0000003	total: 200ms	remaining: 684ms
226:	learn: 0.0000003	total: 201ms	remaining: 683ms
227:	learn: 0.0000003	total: 202ms	remaining: 683ms
228:	learn: 0.0000003	total: 203ms	remaining: 682ms
229:	learn: 0.0000003	total: 204ms	remaining: 682ms
230:	learn: 0.0000003	total: 204ms	remaining: 681ms
231:	learn: 0.0000002	total: 205ms	remaining: 680ms
232:	learn: 0.0000002	total: 206ms	remaining: 679ms
233:	learn: 0.0000002	total: 207ms	remaining: 678ms
234:	learn: 0.0000002	total: 208ms	remaining: 677ms
235:	learn: 

425:	learn: 0.0000000	total: 367ms	remaining: 494ms
426:	learn: 0.0000000	total: 368ms	remaining: 493ms
427:	learn: 0.0000000	total: 369ms	remaining: 493ms
428:	learn: 0.0000000	total: 370ms	remaining: 492ms
429:	learn: 0.0000000	total: 371ms	remaining: 492ms
430:	learn: 0.0000000	total: 372ms	remaining: 491ms
431:	learn: 0.0000000	total: 373ms	remaining: 491ms
432:	learn: 0.0000000	total: 374ms	remaining: 490ms
433:	learn: 0.0000000	total: 375ms	remaining: 489ms
434:	learn: 0.0000000	total: 376ms	remaining: 488ms
435:	learn: 0.0000000	total: 377ms	remaining: 488ms
436:	learn: 0.0000000	total: 378ms	remaining: 487ms
437:	learn: 0.0000000	total: 379ms	remaining: 486ms
438:	learn: 0.0000000	total: 380ms	remaining: 486ms
439:	learn: 0.0000000	total: 381ms	remaining: 485ms
440:	learn: 0.0000000	total: 382ms	remaining: 484ms
441:	learn: 0.0000000	total: 383ms	remaining: 484ms
442:	learn: 0.0000000	total: 384ms	remaining: 483ms
443:	learn: 0.0000000	total: 385ms	remaining: 482ms
444:	learn: 

623:	learn: 0.0000000	total: 543ms	remaining: 327ms
624:	learn: 0.0000000	total: 544ms	remaining: 327ms
625:	learn: 0.0000000	total: 545ms	remaining: 326ms
626:	learn: 0.0000000	total: 546ms	remaining: 325ms
627:	learn: 0.0000000	total: 547ms	remaining: 324ms
628:	learn: 0.0000000	total: 548ms	remaining: 323ms
629:	learn: 0.0000000	total: 549ms	remaining: 322ms
630:	learn: 0.0000000	total: 550ms	remaining: 321ms
631:	learn: 0.0000000	total: 550ms	remaining: 321ms
632:	learn: 0.0000000	total: 551ms	remaining: 320ms
633:	learn: 0.0000000	total: 552ms	remaining: 319ms
634:	learn: 0.0000000	total: 553ms	remaining: 318ms
635:	learn: 0.0000000	total: 554ms	remaining: 317ms
636:	learn: 0.0000000	total: 554ms	remaining: 316ms
637:	learn: 0.0000000	total: 555ms	remaining: 315ms
638:	learn: 0.0000000	total: 556ms	remaining: 314ms
639:	learn: 0.0000000	total: 557ms	remaining: 313ms
640:	learn: 0.0000000	total: 558ms	remaining: 312ms
641:	learn: 0.0000000	total: 559ms	remaining: 312ms
642:	learn: 

821:	learn: 0.0000000	total: 719ms	remaining: 156ms
822:	learn: 0.0000000	total: 720ms	remaining: 155ms
823:	learn: 0.0000000	total: 721ms	remaining: 154ms
824:	learn: 0.0000000	total: 722ms	remaining: 153ms
825:	learn: 0.0000000	total: 722ms	remaining: 152ms
826:	learn: 0.0000000	total: 723ms	remaining: 151ms
827:	learn: 0.0000000	total: 724ms	remaining: 150ms
828:	learn: 0.0000000	total: 725ms	remaining: 150ms
829:	learn: 0.0000000	total: 726ms	remaining: 149ms
830:	learn: 0.0000000	total: 726ms	remaining: 148ms
831:	learn: 0.0000000	total: 727ms	remaining: 147ms
832:	learn: 0.0000000	total: 728ms	remaining: 146ms
833:	learn: 0.0000000	total: 729ms	remaining: 145ms
834:	learn: 0.0000000	total: 730ms	remaining: 144ms
835:	learn: 0.0000000	total: 731ms	remaining: 143ms
836:	learn: 0.0000000	total: 731ms	remaining: 142ms
837:	learn: 0.0000000	total: 732ms	remaining: 142ms
838:	learn: 0.0000000	total: 733ms	remaining: 141ms
839:	learn: 0.0000000	total: 734ms	remaining: 140ms
840:	learn: 

...

0:	learn: 0.8449627	total: 718us	remaining: 718ms
1:	learn: 0.7019408	total: 2ms	remaining: 1000ms
2:	learn: 0.5743336	total: 2.85ms	remaining: 947ms
3:	learn: 0.4926581	total: 3.8ms	remaining: 945ms
4:	learn: 0.4390570	total: 4.78ms	remaining: 952ms
5:	learn: 0.3796806	total: 5.68ms	remaining: 941ms
6:	learn: 0.3236099	total: 6.55ms	remaining: 929ms
7:	learn: 0.2880741	total: 7.53ms	remaining: 934ms
8:	learn: 0.2470001	total: 8.5ms	remaining: 936ms
9:	learn: 0.2237368	total: 9.48ms	remaining: 939ms
10:	learn: 0.1989631	total: 10.3ms	remaining: 927ms
11:	learn: 0.1779697	total: 11.3ms	remaining: 930ms
12:	learn: 0.1583642	total: 12.1ms	remaining: 918ms
13:	learn: 0.1418032	total: 13ms	remaining: 916ms
14:	learn: 0.1321439	total: 14.1ms	remaining: 924ms
15:	learn: 0.1219783	total: 14.9ms	remaining: 915ms
16:	learn: 0.1113282	total: 15.8ms	remaining: 913ms
17:	learn: 0.1066694	total: 16.6ms	remaining: 904ms
18:	learn: 0.0987094	total: 17.6ms	remaining: 907ms
19:	learn: 0.0911711	total: 1

194:	learn: 0.0000011	total: 175ms	remaining: 723ms
195:	learn: 0.0000010	total: 176ms	remaining: 723ms
196:	learn: 0.0000010	total: 177ms	remaining: 722ms
197:	learn: 0.0000009	total: 178ms	remaining: 721ms
198:	learn: 0.0000009	total: 179ms	remaining: 720ms
199:	learn: 0.0000008	total: 180ms	remaining: 720ms
200:	learn: 0.0000008	total: 181ms	remaining: 718ms
201:	learn: 0.0000008	total: 182ms	remaining: 717ms
202:	learn: 0.0000007	total: 182ms	remaining: 716ms
203:	learn: 0.0000007	total: 183ms	remaining: 714ms
204:	learn: 0.0000007	total: 184ms	remaining: 713ms
205:	learn: 0.0000007	total: 185ms	remaining: 712ms
206:	learn: 0.0000006	total: 186ms	remaining: 711ms
207:	learn: 0.0000006	total: 186ms	remaining: 709ms
208:	learn: 0.0000006	total: 187ms	remaining: 708ms
209:	learn: 0.0000005	total: 188ms	remaining: 708ms
210:	learn: 0.0000005	total: 189ms	remaining: 707ms
211:	learn: 0.0000005	total: 190ms	remaining: 707ms
212:	learn: 0.0000005	total: 191ms	remaining: 706ms
213:	learn: 

399:	learn: 0.0000000	total: 349ms	remaining: 523ms
400:	learn: 0.0000000	total: 350ms	remaining: 523ms
401:	learn: 0.0000000	total: 351ms	remaining: 522ms
402:	learn: 0.0000000	total: 352ms	remaining: 521ms
403:	learn: 0.0000000	total: 353ms	remaining: 520ms
404:	learn: 0.0000000	total: 353ms	remaining: 519ms
405:	learn: 0.0000000	total: 354ms	remaining: 518ms
406:	learn: 0.0000000	total: 355ms	remaining: 517ms
407:	learn: 0.0000000	total: 356ms	remaining: 516ms
408:	learn: 0.0000000	total: 357ms	remaining: 515ms
409:	learn: 0.0000000	total: 357ms	remaining: 514ms
410:	learn: 0.0000000	total: 358ms	remaining: 514ms
411:	learn: 0.0000000	total: 359ms	remaining: 513ms
412:	learn: 0.0000000	total: 361ms	remaining: 513ms
413:	learn: 0.0000000	total: 362ms	remaining: 512ms
414:	learn: 0.0000000	total: 363ms	remaining: 511ms
415:	learn: 0.0000000	total: 364ms	remaining: 511ms
416:	learn: 0.0000000	total: 365ms	remaining: 510ms
417:	learn: 0.0000000	total: 366ms	remaining: 509ms
418:	learn: 

601:	learn: 0.0000000	total: 523ms	remaining: 346ms
602:	learn: 0.0000000	total: 524ms	remaining: 345ms
603:	learn: 0.0000000	total: 525ms	remaining: 344ms
604:	learn: 0.0000000	total: 526ms	remaining: 343ms
605:	learn: 0.0000000	total: 527ms	remaining: 343ms
606:	learn: 0.0000000	total: 528ms	remaining: 342ms
607:	learn: 0.0000000	total: 529ms	remaining: 341ms
608:	learn: 0.0000000	total: 530ms	remaining: 340ms
609:	learn: 0.0000000	total: 530ms	remaining: 339ms
610:	learn: 0.0000000	total: 531ms	remaining: 338ms
611:	learn: 0.0000000	total: 532ms	remaining: 337ms
612:	learn: 0.0000000	total: 533ms	remaining: 337ms
613:	learn: 0.0000000	total: 534ms	remaining: 336ms
614:	learn: 0.0000000	total: 535ms	remaining: 335ms
615:	learn: 0.0000000	total: 536ms	remaining: 334ms
616:	learn: 0.0000000	total: 537ms	remaining: 333ms
617:	learn: 0.0000000	total: 538ms	remaining: 332ms
618:	learn: 0.0000000	total: 539ms	remaining: 331ms
619:	learn: 0.0000000	total: 539ms	remaining: 331ms
620:	learn: 

809:	learn: 0.0000000	total: 697ms	remaining: 164ms
810:	learn: 0.0000000	total: 698ms	remaining: 163ms
811:	learn: 0.0000000	total: 699ms	remaining: 162ms
812:	learn: 0.0000000	total: 700ms	remaining: 161ms
813:	learn: 0.0000000	total: 701ms	remaining: 160ms
814:	learn: 0.0000000	total: 702ms	remaining: 159ms
815:	learn: 0.0000000	total: 702ms	remaining: 158ms
816:	learn: 0.0000000	total: 703ms	remaining: 158ms
817:	learn: 0.0000000	total: 704ms	remaining: 157ms
818:	learn: 0.0000000	total: 705ms	remaining: 156ms
819:	learn: 0.0000000	total: 706ms	remaining: 155ms
820:	learn: 0.0000000	total: 707ms	remaining: 154ms
821:	learn: 0.0000000	total: 708ms	remaining: 153ms
822:	learn: 0.0000000	total: 709ms	remaining: 152ms
823:	learn: 0.0000000	total: 710ms	remaining: 152ms
824:	learn: 0.0000000	total: 711ms	remaining: 151ms
825:	learn: 0.0000000	total: 711ms	remaining: 150ms
826:	learn: 0.0000000	total: 712ms	remaining: 149ms
827:	learn: 0.0000000	total: 713ms	remaining: 148ms
828:	learn: 

In [31]:
test_id['class'] = preds
test_id

,id,class
0,TEST_000,0
1,TEST_001,1
2,TEST_002,2
3,TEST_003,1
4,TEST_004,0
...,...,...
170,TEST_170,1
171,TEST_171,2
172,TEST_172,2
173,TEST_173,1


In [34]:
# Decoding

for i, j in zip(target_list, range(len(target_list))):
    test_id['class'] = test_id['class'].replace(j, i)
    
test_id

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [35]:
test_id.to_csv('submission_02.csv', index=False, encoding='utf-8')